# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [5]:
%pip install torchmetrics clu ml-collections torch_geometric polars rdkit -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

1.12.1+cu116


In [2]:
%pip show jax | grep Version

Version: 0.4.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!nvidia-smi

Sun Jul 23 15:20:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   36C    P8     9W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## PNA EPCSAFT

In [ ]:
!python train.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7\
--config.checkpoint_every_steps=2000 --config.learning_rate=0.001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.propagation_depth=5 --config.hidden_dim=512 --config.num_mlp_layers=2 --config.pre_layers=2 --config.post_layers=1

x64 on
I0723 15:39:36.638209 140661704505152 train.py:253] JAX host: 0 / 1
I0723 15:39:36.638499 140661704505152 train.py:254] JAX local devices: [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
I0723 15:39:36.639035 140661704505152 train.py:256] Calling train and evaluate
wandb: Currently logged in as: wildsonbbl (wildson). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /notebooks/GNN-ePC-SAFT/wandb/run-20230723_153938-16muml1t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run daily-water-516
wandb: ⭐️ View project at https://wandb.ai/wildson/gnn-pc-saft
wandb: 🚀 View run at https://wandb.ai/wildson/gnn-pc-saft/runs/16muml1t
I0723 15:39:39.165136 140661704505152 train.py:92] Obtaining datasets.
inchis saved: 1940
I0723 15:39:39.204462 140661704505152 train.py:119] Initial

In [ ]:
!python tuner.py --workdir=/workspaces/ePC-SAFT/ --config=configs/default.py \
--config.num_train_steps={int(6000)} --config.learning_rate=0.001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para={int(3)}